## 1. Setup

In [1]:
import yaml
import os
import sys

# Windows compatibility
if sys.platform == "win32":
    import socketserver
    if not hasattr(socketserver, 'UnixStreamServer'):
        socketserver.UnixStreamServer = socketserver.TCPServer

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import tempfile

# Load configuration
with open("../config/config.yml") as f:
    config = yaml.safe_load(f)

silver_dir = config["paths"]["silver_dir"]
print(f"✓ Silver output: {silver_dir}")

✓ Silver output: data/silver


In [2]:
# Initialize Spark
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
local_temp = tempfile.gettempdir()

spark = SparkSession.builder \
    .appName("GenerateMockParquet") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "localhost") \
    .config("spark.ui.enabled", "false") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config("spark.local.dir", local_temp) \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print(f"✓ Spark version: {spark.version}")
print(f"✓ HADOOP_HOME: {os.environ.get('HADOOP_HOME', 'Not set')}")

✓ Spark version: 3.5.3
✓ HADOOP_HOME: C:\hadoop


## 2. Définir les Schémas (selon schemas.md)

In [3]:
# Schema silver_amenagements
schema_amenagements = StructType([
    StructField("amenagement_id", StringType(), False),
    StructField("annee_livraison", IntegerType(), True),
    StructField("type_amenagement", StringType(), True),
    StructField("environnement", StringType(), True),
    StructField("longueur_m", FloatType(), True),
    StructField("geom_wkt", StringType(), True),
    StructField("centroid_lat", FloatType(), True),
    StructField("centroid_lon", FloatType(), True),
    StructField("commune", StringType(), True)
])

# Schema silver_sites
schema_sites = StructType([
    StructField("site_id", StringType(), False),
    StructField("lat", FloatType(), True),
    StructField("lon", FloatType(), True),
    StructField("commune", StringType(), True)
])

# Schema silver_channels
schema_channels = StructType([
    StructField("channel_id", StringType(), False),
    StructField("site_id", StringType(), True),
    StructField("mode", StringType(), True),
    StructField("sens", StringType(), True)
])

# Schema silver_measures
schema_measures = StructType([
    StructField("channel_id", StringType(), True),
    StructField("date", DateType(), True),
    StructField("flux", IntegerType(), True),
    StructField("is_valid", BooleanType(), True)
])

print("✓ Schemas définis selon schemas.md")

✓ Schemas définis selon schemas.md


## 3. Créer les Données Mock

In [4]:
# Mock silver_amenagements (3 infrastructures)
# Note: Sites sont à ~50-150m des amenagements pour tester le buffer
data_amenagements = [
    ("AMEN_001", 2020, "Piste cyclable", "Urbain", 500.0, 
     "LINESTRING(4.835 45.764, 4.836 45.765)", 45.764, 4.835, "Lyon"),
    ("AMEN_002", 2021, "Bande cyclable", "Périurbain", 300.0,
     "LINESTRING(4.840 45.770, 4.841 45.771)", 45.770, 4.840, "Villeurbanne"),
    ("AMEN_003", 2019, "Voie verte", "Urbain", 800.0,
     "LINESTRING(4.850 45.750, 4.851 45.751)", 45.750, 4.850, "Lyon"),
]

df_amenagements = spark.createDataFrame(data_amenagements, schema_amenagements)
print(f"✓ Created silver_amenagements ({len(data_amenagements)} rows)")

✓ Created silver_amenagements (3 rows)


In [5]:
# Mock silver_sites (3 compteurs)
# SITE_001: ~100m de AMEN_001
# SITE_002: ~80m de AMEN_002  
# SITE_003: >500m de tous (hors buffer)
data_sites = [
    ("SITE_001", 45.7648, 4.8358, "Lyon"),       # ~100m de AMEN_001
    ("SITE_002", 45.7706, 4.8408, "Villeurbanne"),  # ~80m de AMEN_002
    ("SITE_003", 45.780, 4.860, "Villeurbanne"),    # Loin de tout
]

df_sites = spark.createDataFrame(data_sites, schema_sites)
print(f"✓ Created silver_sites ({len(data_sites)} rows)")

✓ Created silver_sites (3 rows)


In [6]:
# Mock silver_channels (5 canaux)
data_channels = [
    ("CHAN_001", "SITE_001", "velo", "Nord"),
    ("CHAN_002", "SITE_001", "velo", "Sud"),
    ("CHAN_003", "SITE_002", "velo", "Est"),
    ("CHAN_004", "SITE_003", "velo", "Ouest"),
    ("CHAN_005", "SITE_002", "voiture", "Nord"),  # Non-vélo, sera filtré
]

df_channels = spark.createDataFrame(data_channels, schema_channels)
print(f"✓ Created silver_channels ({len(data_channels)} rows)")

✓ Created silver_channels (5 rows)


In [7]:
# Mock silver_measures (30 jours de données)
from datetime import date, timedelta

base_date = date(2023, 6, 1)
data_measures = []

for day in range(30):
    current_date = base_date + timedelta(days=day)
    # CHAN_001: 150-210 vélos/jour
    data_measures.append(("CHAN_001", current_date, 150 + day * 2, True))
    # CHAN_002: 100-130 vélos/jour
    data_measures.append(("CHAN_002", current_date, 100 + day, True))
    # CHAN_003: 250-340 vélos/jour
    data_measures.append(("CHAN_003", current_date, 250 + day * 3, True))
    # CHAN_004: 75-105 vélos/jour (hors buffer, ne sera pas utilisé)
    data_measures.append(("CHAN_004", current_date, 75 + day, True))

df_measures = spark.createDataFrame(data_measures, schema_measures)
print(f"✓ Created silver_measures ({len(data_measures)} rows)")

✓ Created silver_measures (120 rows)


## 4. Sauvegarder en Parquet

In [8]:
# Create output directory
output_path = f"../{silver_dir}"
os.makedirs(output_path, exist_ok=True)

# Save as Parquet
df_amenagements.write.mode("overwrite").parquet(f"{output_path}/silver_amenagements")
print(f"✓ Saved silver_amenagements.parquet")

df_sites.write.mode("overwrite").parquet(f"{output_path}/silver_sites")
print(f"✓ Saved silver_sites.parquet")

df_channels.write.mode("overwrite").parquet(f"{output_path}/silver_channels")
print(f"✓ Saved silver_channels.parquet")

df_measures.write.mode("overwrite").parquet(f"{output_path}/silver_measures")
print(f"✓ Saved silver_measures.parquet")

print(f"\n🎉 Toutes les données Silver sont sauvegardées en Parquet dans {output_path}/")

Py4JJavaError: An error occurred while calling o93.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 0.0 failed 1 times, most recent failure: Lost task 6.0 in stage 0.0 (TID 6) (10.42.235.89 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.colle

In [ ]:
# Verify saved files
print("=== Vérification des fichiers Parquet ===")
for table in ["silver_amenagements", "silver_sites", "silver_channels", "silver_measures"]:
    df = spark.read.parquet(f"{output_path}/{table}")
    print(f"{table}: {df.count()} rows, {len(df.columns)} columns")

In [ ]:
# Stop Spark
spark.stop()
print("✓ Spark session stopped")

## 5. Structure des Fichiers Générés

```
data/silver/
├── silver_amenagements/
│   └── *.parquet
├── silver_sites/
│   └── *.parquet
├── silver_channels/
│   └── *.parquet
└── silver_measures/
    └── *.parquet
```

**Prochaine étape :** Exécuter `02_spatial_usage_parquet.ipynb` pour le traitement Module 2